In [1]:
import psi4
import numpy as np
from glob import glob
from scf_guess.io import load_molecule
from scf_guess.metrics import f_score

In [2]:
psi4.set_memory("2 GB")
psi4.set_num_threads(8)
psi4.core.be_quiet()


  Memory set to   1.863 GiB by Python driver.
  Threads set to 8 by Python driver.


In [3]:
xyz_paths = glob("../data/test_data/geometries/**/*.xyz")
print(f"Found {len(xyz_paths)} molecules")

Found 261 molecules


In [4]:
singlets = []
nonsinglets = []

for i, xyz_path in enumerate(xyz_paths):
    mol = load_molecule(xyz_path)
    if not mol.molecular_charge() == 0:
        print(f"{xyz_path} skipped because not charge neutral.")
        continue
    if mol.multiplicity() == 1:
        singlets.append(xyz_path)
    else:
        nonsinglets.append(xyz_path)

assert len(singlets) == 222
assert len(nonsinglets) == 37

../data/test_data/geometries/vanLenthe2006/CrO4_2-.xyz skipped because not charge neutral.
../data/test_data/geometries/vanLenthe2006/Co(NH3)6.xyz skipped because not charge neutral.


In [5]:
def calculate_guess(mol, guess="SAD"):
    try:
        psi4.set_options({"basis": "pcseg-0", "GUESS": guess})
        # Build the guess density
        basis = psi4.core.BasisSet.build(
            mol, target=psi4.core.get_global_option("BASIS")
        )
        ref_wfn = psi4.core.Wavefunction.build(mol, basis)
        start_wfn = psi4.driver.scf_wavefunction_factory(
            name="hf",
            ref_wfn=ref_wfn,
            reference="RHF" if mol.multiplicity == 1 else "UHF",
        )
        start_wfn.form_H()
        start_wfn.form_Shalf()
        start_wfn.guess()
    finally:
        psi4.core.clean()
    return start_wfn.Da_subset("AO").np, start_wfn.Db_subset("AO").np

In [6]:
def load_reference(path):
    wfn = psi4.core.Wavefunction.from_file(path)
    Da, Db = wfn.Da_subset("AO").np, wfn.Db_subset("AO").np
    S = psi4.core.Matrix(*Da.shape)
    S.remove_symmetry(wfn.S(), wfn.aotoso().transpose())
    return Da, Db, S

In [7]:
def score_data_set(paths, guess="SAD"):
    f_scores = np.empty(len(paths))
    f_scores.fill(np.nan)

    for i, xyz_path in enumerate(paths):
        mol = load_molecule(xyz_path)
        s = xyz_path.split("/")
        subset = s[-2]
        name = s[-1].removesuffix(".xyz")
        wfn_path = f"../data/test_data/wavefunctions/HF/pcseg-0/{subset}/{name}.npy"
        Da_guess, Db_guess = calculate_guess(mol, guess=guess)
        Da_scf, Db_scf, S = load_reference(wfn_path)
        f_scores[i] = f_score(S, Da_scf, Da_guess, Db_scf, Db_guess)
    return f_scores

| pcseg-0 | singlets | singlets | nonsinglets | nonsinglets |
| ------- | -------- | -------- | ----------- | ----------- | 
| guess   | min $f$  | mean $f$ | min $f$     | mean $f$    |
| GWH     | 0.405    | 0.587    | 0.458       | 0.558       | 
| CORE    | 0.523    | 0.680    | 0.557       | 0.662       | 
| SAD     | 0.711    | 0.908    | 0.739       | 0.871       | 
| SADNO   | 0.758    | 0.973    | 0.861       | 0.959       |
| HUCKEL  | 0.950    | 0.979    | 0.868       | 0.974       |
| CHA-X   | 0.897    | 0.980    | 0.843       | 0.976       |

In [8]:
# WARNING: this takes quite long
guesses = ["GWH", "CORE", "SAD", "SADNO", "HUCKEL", "SAP"]
for guess in guesses:
    f_singlets = score_data_set(singlets, guess=guess)
    f_nonsinglets = score_data_set(nonsinglets, guess=guess)
    print(
        f"{guess:>8s}:",
        f"{np.nanmin(f_singlets):.3f}  {np.nanmean(f_singlets):.3f}  ",
        f"{np.nanmin(f_nonsinglets):.3f}  {np.nanmean(f_nonsinglets):.3f}",
    )

     GWH: 0.742  0.906   0.800  0.898
    CORE: 0.523  0.680   0.550  0.663
     SAD: 0.711  0.908   0.739  0.871
   SADNO: 0.651  0.973   0.858  0.956
  HUCKEL: 0.950  0.981   0.885  0.972
     SAP: 0.876  0.972   0.923  0.980
